In [3]:
%load_ext Cython
%time
%cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


UsageError: Line magic function `%cython` not found (But cell magic `%%cython` exists, did you mean that instead?).


In [4]:
#import needed packages
from gensim.models import doc2vec

import re
import pandas as pd
import numpy as np
import copy

# nltk processing
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier


In [72]:
data = pd.read_csv("recTitleIngredRating.csv") 
dfTitle = data['title']
dfIngred = data['ingred']

In [74]:

def clean_data(ds):
    ds = ds.str.replace('tablespoons', '')
    ds = ds.str.replace('tablespoon', '')
    ds = ds.str.replace('teaspoon', '')
    ds = ds.str.replace('teaspoons', '')
    ds = ds.str.replace('cup', '')
    ds = ds.str.replace('cups', '')
    ds = ds.str.replace('ounce', '')
    ds = ds.str.replace('ounces', '')
    ds = ds.str.replace('pound', '')
    ds = ds.str.replace('pounds', '')
    ds = ds.str.replace('pinch', '')
    ds = ds.str.replace('pinches', '')
    ds = ds.str.replace('taste', '')
    ds = ds.str.replace('dash', '')
    ds = ds.str.replace('dashes', '')


    ds = ds.str.replace('1/8', '')
    ds = ds.str.replace('1/4', '')
    ds = ds.str.replace('1/3', '')
    ds = ds.str.replace('1/2', '')
    ds = ds.str.replace('2/3', '')
    ds = ds.str.replace('1', '')
    ds = ds.str.replace('1', '')
    ds = ds.str.replace('2', '')
    ds = ds.str.replace('3', '')
    ds = ds.str.replace('4', '')
    ds = ds.str.replace('5', '')
    ds = ds.str.replace('6', '')
    ds = ds.str.replace('7', '')
    ds = ds.str.replace('8', '')
    ds = ds.str.replace('9', '')
    ds = ds.str.replace('0', '')
    ds = ds.str.replace('(', '')
    ds = ds.str.replace(')', '')
    ds = ds.str.replace(' s ', '')
    ds = ds.str.replace("'", '')
    ds = ds.str.replace('.', '')
    ds = ds.str.replace(' to ', '')
    ds = ds.str.replace('[', '')
    ds = ds.str.replace(']', '')
    ds = ds.str.replace('"', '')

    ds = ds.str.replace(' or ', '')
    ds = ds.str.replace(' as ', '')
    ds = ds.str.replace(' u ', '')
    return ds
    
data['ingred'] = clean_data(data['ingred'])
data['title'] = clean_data(data['title'])



In [75]:
stopwrds = stopwords.words('english')
# aux function to clean up text
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = re.sub('[^\w\s]',' ', sentence)
    sentence = re.sub('_',' ', sentence)
    sentence = re.sub('\d+',' ', sentence)
    cleaned = ' '.join([w for w in sentence.split() if not w in stopwrds])
    cleaned = ' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    cleaned = ' '.join([w for w in cleaned.split() if not len(w)<=2 ])
    cleaned = cleaned.strip()
    return cleaned

In [76]:
# clean up review docs, add utf8 encoding
#review_docs['textClean'] = review_docs.apply(lambda row: cleaning_text(row['text'].encode("utf8")), axis=1)


In [77]:
# define a few functions for doc2vec processing
def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

# MyDocs reading from a data frame
class MyDocs(object):
    def __iter__(self):
        dfList = dfIngred.values.tolist()
        for i in range(len(dfList)):
            yield doc2vec.TaggedDocument(words=split_sentence(dfIngred[i]), tags=['%s' % dfIngred[i]])
            
            

In [78]:
# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, vector_size = 200, window = 8, min_count = 5, workers = 4)
model.save("title.model")

In [79]:
# testing similar words
print(model.most_similar(positive=["ground","beef"], negative=["slow"], topn=5))


[('lean', 0.6304351687431335), ('minced', 0.6254710555076599), ('crushed', 0.6162852048873901), ('onion', 0.6086111068725586), ('paprika', 0.6044589877128601)]


/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [80]:
tokens = "ground beef".split()

new_vector = model.infer_vector(tokens)

sims = model.docvecs.most_similar([new_vector]) #gives you top 10 document tags and their cosine similarity

In [81]:
sims

[('  margarine, softened,   peanut butter,   white sugar,   packed brown sugar,  egg,   vanilla extract, /  baking soda,   salt,  /all-purpose flour',
  0.9509450197219849),
 ('  unsweetened almond milk,   frozen blackberries,   frozen blueberries,   coconut butter,   honey,   chia seeds',
  0.9504584074020386),
 ('  frozen pineapple chunks,   frozen mango chunks,   toasted coconut vanilla yogurt,   milk',
  0.950424313545227),
 (' /all-purpose flour,   baking soda,   baking powder,   butter, softened,  white sugar,  egg,   vanilla extract',
  0.9490064382553101),
 ('  fresh lime juice,   white wine vinegar,   salt,   freshly ground black pepper,   ground ginger,   dried basil,   dried thyme,   dried parsley,   hot pepper sauce,   cayenne pepper,   vegetable oil,  fresh swordfish fillets',
  0.9485344886779785),
 ('  soy sauce,   honey,   distilled white vinegar,   ground ginger,   garlic powder,   vegetable oil,  flank steak',
  0.9473416805267334),
 ('all-purpose flour,   salt, bakin

In [82]:
#save a pickle file
import pickle

with open('models/model_ingred_only.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [83]:
data.head()

,title,ingred,rating
0,Wins Shrimp and Spaghetti,"uncooked spaghetti, butter, Creole-style s...",4.41
1,Cheez-It® Fried Tofu Pita Pocket,"package extra-firm tofu, cut into -inch cub...",4.00
2,King Crab Appetizers,"packages refrigerated biscuit dough, pac...",4.37
3,Spinach Basil Pasta Salad,"package bow tie pasta, package spinach l...",4.67
4,Butternut Squash Kugel I,"package frozen butternut squash, cubed, m...",4.17


In [84]:
# get a unique summary corpus
#summary_docs = games[['uniqueID','summary']].append(movies[['uniqueID', 'summary']]).append(tv[['uniqueID','summary']])
#summary_docs.head(5)

In [85]:
# define class for summary Doc2vec

# MyDocs reading from a data frame
class MyDocs_summary(object):
    def __iter__(self):
        for i in range(data.shape[0]):
            yield doc2vec.TaggedDocument(words=split_sentence(data.iloc[i,0]), tags=['%s' % data.iloc[i,1]])
            

In [86]:
# Train the doc2vec model on summary
mydocs_summary = MyDocs_summary()
model_summary = doc2vec.Doc2Vec(mydocs_summary, vector_size = 200, window = 8, min_count = 5, workers = 4)
model_summary.save("summary.model")

In [90]:
print(model_summary.wv.most_similar(positive=["stew", "beef"], negative=["slow"], topn=5))



[('smoothie', 0.9997438192367554), ('zucchini', 0.9997418522834778), ('sauce', 0.9997351169586182), ('shrimp', 0.9997339844703674), ('lemon', 0.9997339248657227)]


In [91]:
# Combining both summary and critic reviews doc2vec :model_summary and model 
summary_ingred_docvecs = np.zeros(shape=(len(model_summary.docvecs),400))
for i in range(len(model_summary.docvecs)):
    summary_ingred_docvecs[i] = np.concatenate((model_summary.docvecs[i],model.docvecs[i]), axis = 0)
    
    

In [94]:
#save pickle file
import pickle
with open('summary_ingred_docvecs.pickle', 'wb') as handle:
    pickle.dump(summary_ingred_docvecs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [95]:
import numpy as np
import pandas as pd
import pickle
import copy
####### TEST REC FROM HERE
# For the app, the code for the recommendation would load a pickle from this cell

#to load a pickle file
with open('summary_ingred_docvecs.pickle', 'rb') as f:
    summary_ingred_docvecs = pickle.load(f)
    
# Auxiliary functions for simple recommendation system 

# Calculate cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

# return top_n values from a list
def top_n_index(l,n):
    return sorted(range(len(l)), key=lambda i: l[i])[-(n+1):-1] #-1 to take off the own product from the returned index list

# return a list of unique_id for a given category["games", "movies","tv"]
def category_id_range(category_list):
    # range of ids for each category
    games_range = range(1,20417)
    movies_range = range(20417, 25887)
    tv_range = range(25887, 27865)
    
    category_range = []
    for i in category_list:
        if i=="games":
            category_range = category_range + games_range
        elif i=="movies":
            category_range = category_range + movies_range
        else:
            category_range = category_range + tv_range
    
    return category_range

def content_recommend(item_id, top_n, inputs):
    input_vec = inputs[item_id - 1]
    
    #compute similarity array
    sim_array = map(lambda v: cossim(input_vec, v), inputs)
    
    # recommendation's index (set to 50 to get enough to filter out later)
    recommendation_index = top_n_index(sim_array, 500)
    
    # recommendation's unique id
    recommendation_unique_id = [i+1 for i in recommendation_index]
    
    # recommendation's cossim values
    recommendation_cossim = [sim_array[i] for i in recommendation_index]
    
    top_products = zip(recommendation_unique_id, recommendation_cossim)
    
    # get the range of unique id for a given category prefered by user
    #category_range = category_id_range(category_list)
    
    result = [i for i in top_products]  # if i[0] in category_range]
    
    return result[-top_n:]